# One-machines Federated Learning
Run all the cells, but first create a conda environment ```conda env create -f environment.yml``` (or manually and then run ```pip install -r requirements.txt```).


In [ ]:
# Parameters
n_clients = 10
n_attackers = 1
attack_type = "DP_inverted_loss" # Choose between: 'DP_flip', 'DP_inverted_loss', 'MP_noise', 'MP_gradient'
n_rounds = 50
data_type = "random" # Choose between: '2cluster', 'random', (i.e., non-IID and IID) -- synthetic can only go with random (already non-IID)
dataset = "synthetic" # Choose between 'diabetes', 'breast', 'synthetic', 'mnist', 'cifar10'

## Data creation

In [1]:
import os
 
# Get the current working directory, remove last, and change the current working directory to the parent directory
current_path = os.getcwd()
parent_path = os.path.dirname(current_path) +'/data'
os.chdir(parent_path)
 
# Create CIFAR-10 dataset
print("\033[93m Create CIFAR-10\033[00m")
script_path = os.path.join(parent_path, 'cifar_creation.py')
!python $script_path
 
# Create MNIST dataset
print("\033[93m Create MNIST\033[00m")
script_path = os.path.join(parent_path, 'mnist_creation.py')
!python $script_path
 
# Split client datasets
parent_path = os.path.dirname(current_path)
os.chdir(parent_path)
print("\033[93m Split client datasets\033[00m")
script_path = os.path.join(parent_path, 'data/client_split.py')
!python $script_path --seed 2 --n_clients $n_clients

 Create CIFAR-10
Files already downloaded and verified
Using device: mps
100%|█████████████████████████████████████████| 782/782 [01:06<00:00, 11.72it/s]
 Create MNIST
/Users/dariofenoglio/miniforge3/envs/CF_FL/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dariofenoglio/miniforge3/envs/CF_FL/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using device: mps
100%|█████████████████████████████████████████| 157/157 [00:02<00:00, 76.24it/s]
 Split client datasets


Data creation
Number 

## Training

In [3]:
import subprocess
import time

# Fixed parameters for this example
model = "net"
pers = "0"

# Start server process
server_command = [
    "python", "server_FBPs.py", 
    "--rounds", str(n_rounds), 
    "--data_type", data_type, 
    "--dataset", dataset, 
    "--model", model, 
    "--pers", pers, 
    "--n_clients", str(n_clients), 
    "--n_attackers", str(n_attackers), 
    "--attack_type", attack_type
]
server_process = subprocess.Popen(server_command)
time.sleep(2)

client_processes = []
for i in range(n_clients):
    print("Start Client", i+1)
    # Start client process
    client_command = [
        "python", "client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset
    ]
    client_processes.append(subprocess.Popen(client_command))
    
    
client_malicious_processes = []
for i in range(n_attackers):
    print("Start Attacker", i+1)
    # Start client process
    client_command = [
        "python", "malicious_client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset, 
        "--attack_type", attack_type
    ]
    client_malicious_processes.append(subprocess.Popen(client_command))
    

server_process.wait()
for p in client_processes:
    p.wait()
for p in client_malicious_processes:
    p.wait()



------- Federated Behavioural Planes (FBPs) -------

Used Size Server-Test Set: torch.Size([300, 2])


INFO flwr 2024-08-29 10:14:32,737 | app.py:163 | Starting Flower server, config: ServerConfig(num_rounds=50, round_timeout=None)
INFO flwr 2024-08-29 10:14:32,753 | app.py:176 | Flower ECE: gRPC server running (50 rounds), SSL is disabled
INFO flwr 2024-08-29 10:14:32,753 | server.py:89 | Initializing global parameters
INFO flwr 2024-08-29 10:14:32,753 | server.py:276 | Requesting initial parameters from one random client


Start Client 1
Start Client 2
Start Client 3
Start Client 4
Start Client 5
Start Attacker 1
The file 'metrics.csv' has been deleted.
The file 'metrics.csv' has been deleted.
The file 'metrics.csv' has been deleted.
The file 'metrics.csv' has been deleted.
The file 'metrics.csv' has been deleted.
MPS is available
MPS is available
MPS is available
MPS is available
MPS is available
MPS is available


INFO flwr 2024-08-29 10:14:36,178 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
INFO flwr 2024-08-29 10:14:36,178 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
INFO flwr 2024-08-29 10:14:36,178 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
INFO flwr 2024-08-29 10:14:36,178 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-08-29 10:14:36,179 | connection.py:42 | ChannelConnectivity.IDLE
INFO flwr 2024-08-29 10:14:36,179 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-08-29 10:14:36,179 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2024-08-29 10:14:36,179 | connection.py:42 | ChannelConnectivity.IDLE
INFO flwr 2024-08-29 10:14:36,179 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-08-29 10:14:36,179 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr

Saving round 1 aggregated_parameters...


...

DEBUG flwr 2024-08-29 10:15:05,693 | server.py:173 | evaluate_round 49: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-08-29 10:15:05,720 | server.py:187 | evaluate_round 49 received 6 results and 0 failures
DEBUG flwr 2024-08-29 10:15:05,720 | server.py:222 | fit_round 50: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-08-29 10:15:05,796 | server.py:236 | fit_round 50 received 6 results and 0 failures


Saving round 50 aggregated_parameters...
Training time: 0.56 minutes


DEBUG flwr 2024-08-29 10:15:06,262 | server.py:173 | evaluate_round 50: strategy sampled 6 clients (out of 6)
DEBUG flwr 2024-08-29 10:15:06,286 | server.py:187 | evaluate_round 50 received 6 results and 0 failures
INFO flwr 2024-08-29 10:15:06,286 | server.py:153 | FL finished in 30.067417416954413
INFO flwr 2024-08-29 10:15:06,286 | app.py:226 | app_fit: losses_distributed [(1, 0.8437466347592106), (2, 0.8621650466681302), (3, 0.8744808284329696), (4, 0.8716141648049732), (5, 0.8625439542990464), (6, 0.8519806848962025), (7, 0.8420288720988504), (8, 0.839792576345049), (9, 0.8362894898375349), (10, 0.8355348378193856), (11, 0.8352424563720967), (12, 0.8346208196419936), (13, 0.8344809516925337), (14, 0.835017722192849), (15, 0.831198361557974), (16, 0.8310036095482815), (17, 0.8294625602589915), (18, 0.832388522888444), (19, 0.8306674177830036), (20, 0.826729229379966), (21, 0.8233526945114136), (22, 0.8261288310827956), (23, 0.8178512347480716), (24, 0.8193627296909609), (25, 0.8157


Client 4 
Minimum Loss occurred at round 50 with a loss value of 0.3727591633796692 
Maximum Accuracy occurred at round 48 with an accuracy value of 1.0 
Validity occurred at round 47 with a validity value of 0.4595959484577179


Client 1 
Minimum Loss occurred at round 48 with a loss value of 0.606674075126648 
Maximum Accuracy occurred at round 48 with an accuracy value of 0.7348484992980957 
Validity occurred at round 50 with a validity value of 0.439393937587738


Client 5 
Minimum Loss occurred at round 50 with a loss value of 0.3918327987194061 
Maximum Accuracy occurred at round 48 with an accuracy value of 0.9769230484962464 
Validity occurred at round 50 with a validity value of 0.4384615421295166


Client 3 
Minimum Loss occurred at round 50 with a loss value of 0.4406557977199554 
Maximum Accuracy occurred at round 48 with an accuracy value of 1.0 
Validity occurred at round 45 with a validity value of 0.4455445408821106


Client 2 
Minimum Loss occurred at round 1 with a l

100%|██████████| 50/50 [00:05<00:00,  9.98it/s]
